# KUSTOMER FETCH DATA

In [2]:
!pip install pytz

In [5]:
import requests
import pandas as pd
from datetime import datetime, date, timedelta
import pytz
import os

# Constants
KUSTOMER_API_URL = "https://edoofa.api.kustomerapp.com/v1/customers"
AUTH_TOKEN = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpZCI6IjY1MGVjMDVkNjg4MGQwNmJlNTNjZWVhMyIsInVzZXIiOiI2NTBlYzA1YmEzODAxZTQ4YjAwOGNhMDQiLCJvcmciOiI2M2JiZGEzNzY4ODFhYTZlMzdmZGRjNmMiLCJvcmdOYW1lIjoiZWRvb2ZhIiwidXNlclR5cGUiOiJtYWNoaW5lIiwicG9kIjoicHJvZDIiLCJyb2xlcyI6WyJvcmcucGVybWlzc2lvbi5tZXNzYWdlLnJlYWQiLCJvcmcucGVybWlzc2lvbi5ub3RlLnJlYWQiLCJvcmcudXNlci5jdXN0b21lci5yZWFkIiwib3JnLnVzZXIubWVzc2FnZS5yZWFkIiwib3JnLnVzZXIubm90ZS5yZWFkIiwib3JnLnBlcm1pc3Npb24uY3VzdG9tZXIucmVhZCJdLCJhdWQiOiJ1cm46Y29uc3VtZXIiLCJpc3MiOiJ1cm46YXBpIiwic3ViIjoiNjUwZWMwNWJhMzgwMWU0OGIwMDhjYTA0In0.IJI5P-BtBDCda9faVA3gfUYHA_rOZnWYuGM0np2Fbng"  # Replace with your actual API token
HEADERS = {
    "Authorization": f"Bearer {AUTH_TOKEN}",
    "Content-Type": "application/json"
}

mentor_mapping = {
    "64e5b276a184b56000839253": "Sharda Nayak",
    "63c01cb8bb84f3f762184233": "Saloni Rastogi",
    "645cc7e93d165050197cc502": "Ananya",
    "63c01c92a204a608c19c95cb": "Tenzin",
    "640e1696673d90d1182aadc6": "Jasmine Kaur",
    "63beab6b88e9f12575db3466": "Aditi Kapoor"
}

def fetch_customer_messages(customer_id):
    message_endpoint = f"{KUSTOMER_API_URL}/{customer_id}/messages"
    page = 1
    all_messages = []
    while True:
        response = requests.get(f"{message_endpoint}?page={page}&pageSize=1000", headers=HEADERS)
        if response.status_code != 200:
            break
        message_data = response.json()
        all_messages.extend(message_data['data'])
        if len(message_data['data']) < 1000:
            break
        page += 1
    return all_messages

def analyze_chats(chat_df, analysis_date):
    analysis_data = []
    unique_students = chat_df['Student'].unique()
    
    for student in unique_students:
        student_chats = chat_df[(chat_df['Student'] == student) & (chat_df['Date'].dt.date == analysis_date)]
        
        if student_chats.empty:
            missed_reply_mentor = 'No'
            missed_reply_student = 'No'
            broken_chat_mentor = 'No'
            broken_chat_student = 'No'
        else:
            # Check if mentor replied
            mentor_replied = not student_chats[student_chats['Direction'] == 'out'].empty
            # Check if student replied
            student_replied = not student_chats[student_chats['Direction'] == 'in'].empty

            # Determine missed replies
            missed_reply_mentor = 'Yes' if student_replied and not mentor_replied else 'No'
            missed_reply_student = 'Yes' if mentor_replied and not student_replied else 'No'

            # Determine broken chats
            last_message = student_chats.iloc[-1]
            broken_chat_mentor = 'Yes' if last_message['Direction'] == 'in' and mentor_replied else 'No'
            broken_chat_student = 'Yes' if last_message['Direction'] == 'out' and student_replied else 'No'

        analysis_data.append([student, missed_reply_mentor, missed_reply_student, broken_chat_mentor, broken_chat_student])

    return pd.DataFrame(analysis_data, columns=['Student', 'Missed Reply by Mentor', 'Missed Reply by Student', 'Broken Chat by Mentor', 'Broken Chat by Student'])

def main():
    # Set the date range for filtering
    # Define the current date in the 'Asia/Kolkata' timezone
    current_date = datetime.now(pytz.timezone("Asia/Kolkata"))

    # Define start date as the start of the day three days ago
    start_date = current_date - timedelta(days=1)
    start_date = start_date.replace(hour=0, minute=0, second=0, microsecond=0)

    # Define end date as the end of the day yesterday
    end_date = current_date  #timedelta(days=0)
    end_date = end_date.replace(hour=0, minute=0, second=0, microsecond=0)

    # Define the analysis date as two days ago
    analysis_date = current_date.date() - timedelta(days=1)

    response = requests.get(KUSTOMER_API_URL, headers=HEADERS)
    if response.status_code != 200:
        print("Failed to fetch customer data")
        return

    customers = response.json()['data']
    all_rows = []

    for customer in customers:
        customer_id = customer['id']
        first_name = customer['attributes'].get('firstName', 'Unknown')
        last_name = customer['attributes'].get('lastName', 'Unknown')
        full_name = f"{first_name} {last_name}"

        messages = fetch_customer_messages(customer_id)
        for message in messages:
            sent_at_utc = datetime.strptime(message['attributes']['sentAt'], '%Y-%m-%dT%H:%M:%S.%fZ').replace(tzinfo=pytz.utc)
            sent_at_ist = sent_at_utc.astimezone(pytz.timezone("Asia/Kolkata"))

            # Filter messages based on the date range
            if start_date <= sent_at_ist <= end_date:
                sent_at_local = sent_at_ist.strftime("%m/%d/%Y %H:%M:%S")
                direction = message['attributes']['direction']
                preview = message['attributes']['preview']
                user_id = message['relationships'].get('createdBy', {}).get('data', {}).get('id', None)
                mentor_name = mentor_mapping.get(user_id, "Unknown Mentor")

                all_rows.append([sent_at_local, preview, mentor_name, full_name, direction])

    chat_df = pd.DataFrame(all_rows, columns=['Date', 'Body', 'Mentor', 'Student', 'Direction'])
    chat_df['Date'] = pd.to_datetime(chat_df['Date'])
    analysis_df = analyze_chats(chat_df, analysis_date)

    # Save as Excel with multiple sheets
    try:
        base_dir = os.path.dirname(os.path.abspath(__file__))
    except NameError:
        base_dir = os.getcwd()
    excel_file_path = os.path.join(base_dir, 'kustomer_analysis15.xlsx')
    with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
        chat_df.to_excel(writer, sheet_name='Chat Data', index=False)
        analysis_df.to_excel(writer, sheet_name='Analysis', index=False)

    print(f"Analysis saved to {excel_file_path}")

if __name__ == "__main__":
    main()

2023-12-13 10:28:51,065 - DEBUG - Starting new HTTPS connection (1): edoofa.api.kustomerapp.com:443
2023-12-13 10:28:52,066 - DEBUG - https://edoofa.api.kustomerapp.com:443 "GET /v1/customers HTTP/1.1" 200 None
2023-12-13 10:28:52,082 - DEBUG - Starting new HTTPS connection (1): edoofa.api.kustomerapp.com:443
2023-12-13 10:28:52,931 - DEBUG - https://edoofa.api.kustomerapp.com:443 "GET /v1/customers/652cd85727b3577e804cb74e/messages?page=1&pageSize=1000 HTTP/1.1" 200 None
2023-12-13 10:28:52,943 - DEBUG - Starting new HTTPS connection (1): edoofa.api.kustomerapp.com:443
2023-12-13 10:28:53,803 - DEBUG - https://edoofa.api.kustomerapp.com:443 "GET /v1/customers/655727223a5d05de5fa3ff54/messages?page=1&pageSize=1000 HTTP/1.1" 200 None
2023-12-13 10:28:53,823 - DEBUG - Starting new HTTPS connection (1): edoofa.api.kustomerapp.com:443
2023-12-13 10:28:54,727 - DEBUG - https://edoofa.api.kustomerapp.com:443 "GET /v1/customers/6528a2e5bd885ff52f8969b3/messages?page=1&pageSize=1000 HTTP/1.1" 

2023-12-13 10:29:21,115 - DEBUG - https://edoofa.api.kustomerapp.com:443 "GET /v1/customers/65682be14ae58a7ee1768fb6/messages?page=1&pageSize=1000 HTTP/1.1" 200 None
2023-12-13 10:29:21,132 - DEBUG - Starting new HTTPS connection (1): edoofa.api.kustomerapp.com:443
2023-12-13 10:29:22,035 - DEBUG - https://edoofa.api.kustomerapp.com:443 "GET /v1/customers/643797eccee32f7e7dfd4b52/messages?page=1&pageSize=1000 HTTP/1.1" 200 None
2023-12-13 10:29:22,209 - DEBUG - Starting new HTTPS connection (1): edoofa.api.kustomerapp.com:443
2023-12-13 10:29:22,997 - DEBUG - https://edoofa.api.kustomerapp.com:443 "GET /v1/customers/65264335d9170e3378d1f01a/messages?page=1&pageSize=1000 HTTP/1.1" 200 None
2023-12-13 10:29:23,020 - DEBUG - Starting new HTTPS connection (1): edoofa.api.kustomerapp.com:443
2023-12-13 10:29:23,813 - DEBUG - https://edoofa.api.kustomerapp.com:443 "GET /v1/customers/65789600bcb38263425c9ed6/messages?page=1&pageSize=1000 HTTP/1.1" 200 None
2023-12-13 10:29:23,813 - DEBUG - St

2023-12-13 10:29:48,297 - DEBUG - https://edoofa.api.kustomerapp.com:443 "GET /v1/customers/655dee8a1b77e8504c684e06/messages?page=1&pageSize=1000 HTTP/1.1" 200 None
2023-12-13 10:29:48,306 - DEBUG - Starting new HTTPS connection (1): edoofa.api.kustomerapp.com:443
2023-12-13 10:29:49,110 - DEBUG - https://edoofa.api.kustomerapp.com:443 "GET /v1/customers/6576cefc1f708154fe3d1d9a/messages?page=1&pageSize=1000 HTTP/1.1" 200 None
2023-12-13 10:29:49,117 - DEBUG - Starting new HTTPS connection (1): edoofa.api.kustomerapp.com:443
2023-12-13 10:29:49,930 - DEBUG - https://edoofa.api.kustomerapp.com:443 "GET /v1/customers/655b142b6b2a0d3cc7b5043c/messages?page=1&pageSize=1000 HTTP/1.1" 200 None
2023-12-13 10:29:49,943 - DEBUG - Starting new HTTPS connection (1): edoofa.api.kustomerapp.com:443
2023-12-13 10:29:50,755 - DEBUG - https://edoofa.api.kustomerapp.com:443 "GET /v1/customers/655b4bf620bf800a77d0954e/messages?page=1&pageSize=1000 HTTP/1.1" 200 None
2023-12-13 10:29:50,762 - DEBUG - St

2023-12-13 10:30:14,404 - DEBUG - https://edoofa.api.kustomerapp.com:443 "GET /v1/customers/651e75e3cc5d1fbe64e6d9ae/messages?page=1&pageSize=1000 HTTP/1.1" 200 None
2023-12-13 10:30:14,556 - DEBUG - Starting new HTTPS connection (1): edoofa.api.kustomerapp.com:443
2023-12-13 10:30:15,437 - DEBUG - https://edoofa.api.kustomerapp.com:443 "GET /v1/customers/643797e8482f807ece3e5723/messages?page=1&pageSize=1000 HTTP/1.1" 200 None
2023-12-13 10:30:15,606 - DEBUG - Starting new HTTPS connection (1): edoofa.api.kustomerapp.com:443
2023-12-13 10:30:16,449 - DEBUG - https://edoofa.api.kustomerapp.com:443 "GET /v1/customers/65771dd58a8faa731a42b220/messages?page=1&pageSize=1000 HTTP/1.1" 200 None
2023-12-13 10:30:16,459 - DEBUG - Starting new HTTPS connection (1): edoofa.api.kustomerapp.com:443
2023-12-13 10:30:17,389 - DEBUG - https://edoofa.api.kustomerapp.com:443 "GET /v1/customers/6437531f23a7c7e0aecff4f6/messages?page=1&pageSize=1000 HTTP/1.1" 200 None
2023-12-13 10:30:17,545 - DEBUG - St

Analysis saved to C:\Users\mauriceyeng\Python\Daily-Reports\kustomer_analysis15.xlsx


# Table 2

In [4]:
import os
import pandas as pd
import re
from datetime import datetime, timedelta
import logging
from datetime import timedelta

# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
# Set of processed chats to ensure each chat is analyzed only once
processed_chats = set()

# Function to process a single line of chat
def process_line(line):
    pattern = r'(\d{2}/\d{2}/\d{2}), (\d{1,2}:\d{2}\s?[APMapm]{2}) - (.*?): (.*)'
    match = re.match(pattern, line)
    if match:
        date_str, time_str, sender, message = match.groups()
        date = datetime.strptime(date_str, '%d/%m/%y')
        time = datetime.strptime(time_str, '%I:%M %p').strftime('%H:%M')
        return {'date': date, 'time': time, 'sender': sender, 'message': message}
    else:
        return None

def process_line_for_delay(line):
    pattern = r'(\d{2}/\d{2}/\d{2}), (\d{1,2}:\d{2}\s?[APMapm]{2}) - (.*?): (.*)'
    match = re.match(pattern, line)
    if match:
        date_str, time_str, sender, message = match.groups()
        datetime_str = date_str + ' ' + time_str
        datetime_obj = datetime.strptime(datetime_str, '%d/%m/%y %I:%M %p')
        return {'datetime': datetime_obj, 'sender': sender, 'message': message}
    else:
        return None

# Function to read the chat file specifically for delay calculation
def read_chat_file_for_delay(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    concatenated_lines = []
    current_message = ""
    date_pattern = re.compile(r'^\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2}\s?[APMapm]{2} - ')

    for line in lines:
        if date_pattern.match(line):
            if current_message:
                concatenated_lines.append(current_message)
            current_message = line.rstrip()
        else:
            # Append this line to the current message, if it's not empty
            if current_message:
                current_message += " " + line.strip()
    if current_message:
        concatenated_lines.append(current_message)

    return pd.DataFrame([process_line_for_delay(line) for line in concatenated_lines if process_line_for_delay(line) is not None])

# The process_line_for_delay function remains the same
       
    
def calculate_time_spent_student(chat_df, target_date, employee_name):
    student_messages = chat_df[(chat_df['date'].dt.date == target_date) & (chat_df['sender'] != employee_name)]
    total_chars = student_messages['message'].str.len().sum()
    time_spent_seconds = (total_chars / 10) * 5
    return strfdelta(timedelta(seconds=time_spent_seconds)) 

def format_delay_time(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    if hours > 0:
        return f"{hours} hr {minutes} mins"
    else:
        return f"{minutes} mins"

def calculate_employee_delay(chat_df, employee_name, target_date):
    delays = []
    delay_times = []
    messages_for_reference = []

    last_student_message_time = None
    employee_responded_after_student = False

    delay_counter = 1

    for index, row in chat_df.iterrows():
        # Ensure datetime is correctly formatted
        row_date = row['datetime'].date()

        if row_date != target_date:
            continue

        if row['sender'] != employee_name:
            last_student_message_time = row['datetime']
            employee_responded_after_student = False
        elif row['sender'] == employee_name and last_student_message_time is not None and not employee_responded_after_student:
            time_diff = row['datetime'] - last_student_message_time

            if time_diff > timedelta(minutes=15):
                formatted_time = row['datetime'].strftime('%d %b %Y %I:%M%p')
                delays.append(f"{delay_counter} - {formatted_time}")
                delay_counter += 1

                delay_times.append(format_delay_time(time_diff.seconds))

                start_index = max(0, index - 5)
                end_index = min(index + 5, len(chat_df))
                context_messages = chat_df.iloc[start_index:end_index]

                formatted_context_messages = [f"{idx+1} - {message_row['datetime'].strftime('%d %b %Y %I:%M%p')} - {message_row['sender']}: {message_row['message']}" for idx, message_row in context_messages.iterrows()]
                formatted_message = "\n".join(formatted_context_messages)
                messages_for_reference.append(formatted_message)

            employee_responded_after_student = True

    # Combine the lists into multiline strings
    delays_str = "\n".join(delays)
    delay_times_str = "\n".join(delay_times)
    messages_for_reference_str = "\n".join(messages_for_reference)

    return delays_str, delay_times_str, messages_for_reference_str

def is_broken_chat_student(chat_df, employee_name, target_date):
    # Adjust for the previous day
    previous_day = target_date - timedelta(days=1)

    # Filter messages for the previous day
    daily_messages = chat_df[chat_df['date'].dt.date == previous_day]

    # Check if both the employee and the student had at least one text on that day
    if daily_messages.empty:
        return 'No'  # No conversation happened on this day

    employee_messages = daily_messages[daily_messages['sender'] == employee_name]
    student_messages = daily_messages[daily_messages['sender'] != employee_name]

    # Check if both parties have sent at least one message
    if employee_messages.empty or student_messages.empty:
        return 'No'  # One of the parties did not send any message

    # Check the sender of the last message of the day
    last_message_sender = daily_messages.iloc[-1]['sender']

    # If the last message is from the employee, it's a broken chat by the student
    if last_message_sender == employee_name:
        return 'Yes'

    return 'No'
    
def count_missed_replies_student(chat_df, employee_name, report_date):
    count = 0
    for i in range(7):
        check_date = report_date - timedelta(days=i)
        messages_on_date = chat_df[chat_df['date'].dt.date == check_date]
        employee_messages = messages_on_date[messages_on_date['sender'] == employee_name]
        student_messages = messages_on_date[messages_on_date['sender'] != employee_name]
        if employee_messages.empty and not student_messages.empty:
            count += 1
    return count

def count_missed_replies_studentt(chat_df, employee_name, target_date):
    previous_day = target_date - timedelta(days=1)
    #print(f"Checking for missed replies on: {previous_day}")

    daily_messages = chat_df[chat_df['date'].dt.date == previous_day]
    
    employee_messages = daily_messages[daily_messages['sender'] == employee_name]
    student_messages = daily_messages[daily_messages['sender'] != employee_name]

    #print(f"Employee messages count: {len(employee_messages)}")
    #print(f"Student messages count: {len(student_messages)}")

    # Check if there are messages from the employee but none from the student
    if not employee_messages.empty and student_messages.empty:
        #print("Missed reply found.")
        return 'Yes'
    
    #print("No missed reply.")
    return 'No'

def count_missed_replies_last_7_days(chat_df, employee_name, start_date):
    missed_replies_count = 0

    # Iterate over the last 7 days
    for i in range(7):
        date_to_check = start_date - timedelta(days=i)
        messages_on_date = chat_df[chat_df['date'].dt.date == date_to_check]

        # Check if there are messages on this date
        if not messages_on_date.empty:
            # Check if there's any message from the employee on this day
            employee_messages = messages_on_date[messages_on_date['sender'] == employee_name]
            
            # Check if there's any message from others (leads) on this day
            lead_messages = messages_on_date[messages_on_date['sender'] != employee_name]

            # If there are messages from others but none from the employee, count as a missed reply
            if not lead_messages.empty and employee_messages.empty:
                missed_replies_count += 1

    return missed_replies_count

# Function to calculate the time spent in chat based on message lengths
def calculate_time_spent(chat_df, target_date, employee_name, team_folder):
    daily_messages = chat_df[(chat_df['date'].dt.date == target_date) & (chat_df['sender'] == employee_name)]

    if daily_messages.empty:
        return '00:00:00'

    total_time_spent_seconds = 0

    for index, row in daily_messages.iterrows():
        message = row['message']
        total_chars = len(message)

        if team_folder == "KAM" and total_chars > 700:
            time_spent_seconds = 5  # Cap at 5 seconds for long messages in KAM team folder
        else:
            time_spent_seconds = total_chars * 0.2  # Assuming 0.2 seconds per character

        total_time_spent_seconds += time_spent_seconds

        # Optional: Debug print
        #print(f"New Chat Found in {team_folder}")
        #print(f"Message Length: '{total_chars}'")
        #print(f"Message: '{message}' - Time Spent: {time_spent_seconds} seconds")

    time_spent_timedelta = timedelta(seconds=total_time_spent_seconds)
    formatted_time_spent = strfdelta(time_spent_timedelta)

    return formatted_time_spent

def strfdelta(tdelta):
    hours, remainder = divmod(tdelta.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours:02}:{minutes:02}:{seconds:02}"   

def calculate_lead_response(chat_df, employee_name, target_date):
    """
    Calculates the lead response based on the number of texts exchanged between the employee and others (non-employee).
    - If only the employee sent messages, the response is 100%.
    - If only non-employees sent messages, the response is 0%.
    - Otherwise, it's the ratio of the employee's texts to the total texts, constrained between 1% and 99%.
    """
    # Filter messages for the target date
    daily_messages = chat_df[chat_df['date'].dt.date == target_date]
    
    # Check if there are any messages from the employee and others on the target date
    employee_messages = daily_messages[daily_messages['sender'] == employee_name]
    non_employee_messages = daily_messages[daily_messages['sender'] != employee_name]

    # If only the employee sent messages
    if not employee_messages.empty and non_employee_messages.empty:
        return 0

    # If only non-employees sent messages
    if employee_messages.empty and not non_employee_messages.empty:
        return 100

    # If both the employee and non-employees have sent messages
    if not employee_messages.empty and not non_employee_messages.empty:
        num_texts_employee = len(employee_messages)
        num_texts_non_employee = len(non_employee_messages)
        total_texts = num_texts_employee + num_texts_non_employee
        lead_response = (num_texts_employee / total_texts) * 100
        return max(min(lead_response, 99), 1)

    # If neither the employee nor non-employees sent messages
    return 0



def calculate_broken_chat_within_working_hrs(chat_df, employee_name, report_date, team_folder):
    # Adjust working hours based on team folder
    if team_folder == 'EWYL':
        # KAM Team working hours: 7:30 AM to 4:00 PM
        work_start_time = datetime(report_date.year, report_date.month, report_date.day, 7, 30, 0)
        work_end_time = datetime(report_date.year, report_date.month, report_date.day, 16, 30, 0)
    else:
        # Default working hours for other teams: 12:00 AM to 12:00 PM
        work_start_time = datetime(report_date.year, report_date.month, report_date.day, 0, 0, 0)
        work_end_time = datetime(report_date.year, report_date.month, report_date.day, 23, 59, 0)

    # Filter the messages for the given report date
    messages_on_date = chat_df[chat_df['datetime'].dt.date == report_date.date()]

    # Check if both employee and lead have messaged on the report date
    if not messages_on_date.empty:
        employee_messages = messages_on_date[messages_on_date['sender'] == employee_name]
        lead_messages = messages_on_date[messages_on_date['sender'] != employee_name]
        last_message_sender = messages_on_date.iloc[-1]['sender']
        # Ensure both the employee and lead have at least one message
        if not employee_messages.empty and not lead_messages.empty:
            # Get the time of the last message from the lead
            last_lead_message_time = lead_messages.iloc[-1]['datetime']

            # Check if the last message is from the lead and within working hours
            if last_message_sender != employee_name and (work_start_time <= last_lead_message_time <= work_end_time):
                return 'Yes'  # Broken chat if last message from the lead is within working hours
            else:
                return 'No'  # Not a broken chat if last message from the lead is outside working hours or from the employee

    # If no messages or only one party messaged, return 'No'
    return 'No'

def calculate_broken_chat(chat_df, employee_name, report_date):
    # Filter the messages for the given report date
    messages_on_date = chat_df[chat_df['date'].dt.date == report_date]
    
    # Check if both employee and lead have messaged on the report date
    if not messages_on_date.empty:
        employee_messages = messages_on_date[messages_on_date['sender'] == employee_name]
        lead_messages = messages_on_date[messages_on_date['sender'] != employee_name]
        
        # Ensure both the employee and lead have at least one message
        if not employee_messages.empty and not lead_messages.empty:
            # Check if the last message is from the lead
            last_message_sender = messages_on_date.iloc[-1]['sender']
            return last_message_sender != employee_name

    

def missed_replies_employee(chat_df, employee_name, target_date):
    # Filter messages for the target date
    daily_messages = chat_df[chat_df['date'].dt.date == target_date]
    
    # Check if there are any messages from the student
    student_messages = daily_messages[daily_messages['sender'] != employee_name]
    
    # Check if there are any messages from the employee
    employee_messages = daily_messages[daily_messages['sender'] == employee_name]
    
    # If there are student messages but no employee messages, consider it a missed reply
    if not student_messages.empty and employee_messages.empty:
        return 'Yes'
    
    return 'No'

def missed_replies_employee_after_working_hrs(chat_df, employee_name, target_date, team_folder):
    # Check if the team folder is 'KAM Team' to define different working hours
    if team_folder == 'EWYL':
        # EWYL Team working hours: 7:30 AM to 4:00 PM
        work_start_time = datetime(target_date.year, target_date.month, target_date.day, 7, 30, 0)
        work_end_time = datetime(target_date.year, target_date.month, target_date.day, 16, 30, 0)
    
    elif team_folder == 'KAM':
       # KAM Team working hours: 7:30 AM to 4:00 PM
        work_start_time = datetime(target_date.year, target_date.month, target_date.day, 11, 30, 0)
        work_end_time = datetime(target_date.year, target_date.month, target_date.day, 21, 30, 0)

    else:
        # Default working hours for other teams: 12:00 AM to 12:00 PM
        work_start_time = datetime(target_date.year, target_date.month, target_date.day, 0, 0, 0)
        work_end_time = datetime(target_date.year, target_date.month, target_date.day, 23, 59, 0)

    # Ensure target_date is a datetime object
    if not isinstance(target_date, datetime):
        raise ValueError("target_date must be a datetime object")

    # Filter messages for the target date using 'datetime' column
    daily_messages = chat_df[chat_df['datetime'].dt.date == target_date.date()]
    
    # Check if there are any messages from the student
    student_messages = daily_messages[daily_messages['sender'] != employee_name]
    
    # Check if there are any messages from the employee
    employee_messages = daily_messages[daily_messages['sender'] == employee_name]

    # If there are student messages but no employee messages
    if not student_messages.empty and employee_messages.empty:
        print(f"ENTER THE LOOP:{student_messages}")
        # Check if any student message is within working hours
        for _, row in student_messages.iterrows():
            message_time = row['datetime']
            if work_start_time <= message_time <= work_end_time:
                return 'Yes'  # Missed reply within working hours
        return 'No'  # All student messages are outside working hours

    return 'No'

# Function to read the chat file and return a dataframe
def read_chat_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    concatenated_lines = []
    current_message = ""
    date_pattern = re.compile(r'^\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2}\s?[APMapm]{2} - ')

    for line in lines:
        if date_pattern.match(line):
            if current_message:
                concatenated_lines.append(current_message)
            current_message = line.rstrip()
        else:
            # Append this line to the current message, if it's not empty
            if current_message:
                current_message += " " + line.strip()
    if current_message:
        concatenated_lines.append(current_message)

    return pd.DataFrame([process_line(line) for line in concatenated_lines if process_line(line) is not None])

# process_line function remains the same


# Function to extract the chat file name based on a specific pattern
def extract_chat_name(chat_file_name):
    #logging.debug(f"Extracting name from chat file: {chat_file_name}")
    match = re.search(r'WhatsApp Chat with _?(.*?)(?:\(\d+\))?_?\.txt', chat_file_name)
    if match:
        extracted_name = re.sub(r'_*(?:\(\d+\))?$', '', match.group(1))
        #logging.debug(f"Extracted chat name: {extracted_name}")
        return extracted_name
    #logging.warning(f"No match found for chat file name: {chat_file_name}")
    return None

# Function to calculate the total count of missed replies by the student for the last N days
def calculate_total_count_missed_replies_student(chat_df, employee_name, num_days):
    total_count = 0

    # Iterate over the last N days
    for i in range(num_days):
        target_date = report_date - timedelta(days=i)
        count = count_missed_replies_student(chat_df, employee_name, target_date)
        total_count += count

    return total_count

# Function to extract the start date from a chat file
def get_chat_start_date(chat_df):
    if not chat_df.empty:
        #logging.debug(f"Extracted chat name: {chat_df['date'].min().date()}")
        return chat_df['date'].min().date()
    return None

# Function to process a single chat file
def process_chat_file(file_path, report_date, main_directory, team_folder, employee_folder):
    global processed_chats
    broken_chat_count = 0
    missed_reply_count = 0
    max_time_spent = ('', '00:00:00')  # (date, time)

    chat_file_name = os.path.basename(file_path)
    #logging.debug(f"Processing chat file: {chat_file_name}")

    chat_name = extract_chat_name(chat_file_name)
    #logging.debug(f"Chat name extracted: {chat_name}")



    

    if chat_name is None:
        #logging.warning(f"Chat name could not be extracted for file: {chat_file_name}")
        return None
    if chat_name in processed_chats:
        #logging.info(f"Chat file already processed: {chat_name}")
        return None

    processed_chats.add(chat_name)
    #logging.debug(f"Chat file added to processed list: {chat_name}")

    chat_df = read_chat_file(file_path)
    if chat_df.empty:
        #logging.warning(f"Chat DataFrame is empty for file: {chat_file_name}")
        return None
    chat_start_date = get_chat_start_date(chat_df)

    # Use read_chat_file_for_delay specifically for delay calculations
    chat_df_for_delay = read_chat_file_for_delay(file_path)
    if chat_df_for_delay.empty:
        return None

    
    target_date = report_date - timedelta(days=1)

    # Call the function with the target date
    delays, delay_times, messages_for_reference = calculate_employee_delay(chat_df_for_delay, employee_folder, target_date)
    



    # Calculate the time spent on Day 0 (report_date - 1 day)
    day_0_date = report_date - timedelta(days=1)
    day_0_time_spent = calculate_time_spent(chat_df, day_0_date, employee_folder,team_folder)
    #day_0_time_spent = timedelta(hours=int(day_0_time_spent.split(':')[0]), minutes=int(day_0_time_spent.split(':')[1]), seconds=int(day_0_time_spent.split(':')[2]))
    # Calculate the lead response based on text length difference for Day 0
    lead_response_day_0 = calculate_lead_response(chat_df, employee_folder, day_0_date)
     
    day_0_date = report_date - timedelta(days=1)  # Assuming report_date is a datetime object of the target date
    broken_chat = calculate_broken_chat(chat_df, employee_folder, day_0_date)
    missed_replies_day_0 = missed_replies_employee(chat_df, employee_folder, day_0_date)

    # Get the current datetime
    today_datetime = datetime.now()

    # Get today's date with time part (assuming start of the day as default)
    today_date_1 = today_datetime.replace(hour=0, minute=0, second=0, microsecond=0)

    # If today_date.weekday() logic remains the same
    # ...

    # Now, when you calculate target_date, it remains a datetime object
    target_date_1 = today_date_1 - timedelta(days=1)


    Actual_missed_Reply = missed_replies_employee_after_working_hrs(chat_df_for_delay, employee_folder, target_date_1,team_folder)
    Actual_broken_chat = calculate_broken_chat_within_working_hrs(chat_df_for_delay, employee_folder, target_date_1,team_folder)
    total_count_missed_reply_student = calculate_total_count_missed_replies_student(chat_df, employee_folder, 14)
    


    
    for i in range(7):  # Assuming you're checking the past 7 days
        check_date = report_date - timedelta(days=i)
        
        # Check for broken chats
        if calculate_broken_chat(chat_df, employee_folder, check_date):
            broken_chat_count += 1

        # Check for missed replies
        start_date_for_missed_replies = report_date - timedelta(days=7)
        
        missed_replies_count = count_missed_replies_last_7_days(chat_df, employee_folder, start_date_for_missed_replies)

        # Check for max time spent
        time_spent = calculate_time_spent(chat_df, check_date, employee_folder,team_folder)
        if time_spent > max_time_spent[1]:
            max_time_spent = (check_date.strftime('%m-%d-%Y'), time_spent)
    
    
    
    # Calculate the time spent by students
    time_spent_student = {
    'Day 0 (Student)': calculate_time_spent_student(chat_df, report_date - timedelta(days=1), employee_folder),
    'Day 1 (Student)': calculate_time_spent_student(chat_df, report_date - timedelta(days=2), employee_folder),
    'Day 2 (Student)': calculate_time_spent_student(chat_df, report_date - timedelta(days=3), employee_folder),
    }
    
    # Inside the process_chat_file function, after calculating day_0_time_spent:
    day_0_date = report_date - timedelta(days=1)
    day_1_date = report_date - timedelta(days=2)
    day_2_date = report_date - timedelta(days=3)

    # Calculate time spent by student for Day 0, Day 1, and Day 2
    time_spent_student_day_0 = calculate_time_spent_student(chat_df, day_0_date, employee_folder)
    time_spent_student_day_1 = calculate_time_spent_student(chat_df, day_1_date, employee_folder)
    time_spent_student_day_2 = calculate_time_spent_student(chat_df, day_2_date, employee_folder)



    total_time_spent_student = time_spent_student['Day 0 (Student)']

    broken_chat_student = is_broken_chat_student(chat_df, employee_folder, report_date)

    mmissed_reply_student = count_missed_replies_studentt(chat_df, employee_folder, report_date)

    # Calculate the time spent on each day relative to the report date
    time_spent = {}
    for i in range(1, 4):  # Start from 1 since we want Day 0 to be the day before the report date
        target_date = report_date - timedelta(days=i)
        time_spent[f'Day {i-1}'] = calculate_time_spent(chat_df, target_date, employee_folder,team_folder)
    
    target_date_for_chat_start_Date = report_date - timedelta(days=1)
    day_indicator_employee = (target_date_for_chat_start_Date - chat_start_date).days
    #logging.debug(f"day indicator employee: {day_indicator_employee}")
    # Determine Day Indicator based on chat start date
    if chat_start_date:
        if (target_date_for_chat_start_Date - chat_start_date).days == 0:
          
            day_indicator = 'Day 0'
        elif (target_date_for_chat_start_Date - chat_start_date).days == 1:
            
            day_indicator = 'Day 1'
        elif (target_date_for_chat_start_Date - chat_start_date).days == 2:
            
            day_indicator = 'Day 2'

        elif (target_date_for_chat_start_Date - chat_start_date).days == 3:
            
            day_indicator = 'OLD'

        else:
            day_indicator = 'OLD'  # For chats that don't fall into Day 0, Day 1, or Day 2

    

    main_directory_name = os.path.basename(main_directory)

    row = {
        'Date': main_directory_name,
        'Team Folder': team_folder,
        'Employee Folder': employee_folder,
        'Chat File Name': chat_name,        # Inserting the extracted chat name
        'Day Indicator': day_indicator,
        'Total_Time_Spent_Employee': day_0_time_spent,  # Use Day 0 time as the total time spent
        'LR': lead_response_day_0,  # Add the lead response here
        'Broken Chat(Employee)': 'Yes' if broken_chat else 'No',
        'Missed Replies (Employee)': missed_replies_day_0,
        'Count_Broken_Chat_Employee (Employee)': broken_chat_count,
        'Count_Missed_Replies_Missed_Replies (Employee)': missed_replies_count,
        'Date of Max time': max_time_spent[0],
        **time_spent,
        'Total Time Spent (Student)': total_time_spent_student,
        'Broken Chat (Student)': broken_chat_student,
        'Missed Replies (Student)': mmissed_reply_student,
        'Day_0_Time_Spent_Student': time_spent_student_day_0,  # Add time spent for Day 0
        'Day_1_Time_Spent_Student': time_spent_student_day_1,  # Add time spent for Day 1
        'Day_2_Time_Spent_Student': time_spent_student_day_2,
        'Total_Count_Missed_Reply_Chat_Student' : total_count_missed_reply_student,
        'Employee Delays': delays,
        'Delay Durations': delay_times,
        'Messages for Reference': messages_for_reference,
        'Actual Missed Reply from Employee': Actual_missed_Reply,
        'Actual Broken Chat from Employee' : Actual_broken_chat,
    }
    #logging.debug(f"Row created for chat: {row}")
    return row

# Main directory path construction and report date setting
# Main directory path construction and report date setting
local_date_format = '%Y-%m-%d'  # Adjust this to your local date format
main_directory = 'C:\\Users\\mauriceyeng\Python\\Daily-Reports\\Chat Folder from Drive\\drive-download-20231213T043134Z-001'  # Replace with your base directory path

# Get today's date
today_date = datetime.now().date()

# Check the day of the week
if today_date.weekday() in range(1, 6):  # Tuesday to Saturday
    report_date = today_date
else:  # Monday
    # You can adjust the number of days to subtract based on your specific requirements
    report_date = today_date - timedelta(days=1)
    logging.debug(f"Report Date: {report_date}")
report_date_str_local = report_date.strftime(local_date_format)
main_directory_path = os.path.join(main_directory, report_date_str_local)

# Function to navigate through the directory structure and process all chat files
def process_team_folders(main_directory, report_date):
    all_data = []
    for team_folder in os.listdir(main_directory):
        team_path = os.path.join(main_directory, team_folder)
        if not os.path.isdir(team_path):
            continue
        for employee_folder in os.listdir(team_path):
            employee_path = os.path.join(team_path, employee_folder)
            if not os.path.isdir(employee_path):
                continue
            for chat_file in os.listdir(employee_path):
                chat_file_path = os.path.join(employee_path, chat_file)
                row = process_chat_file(chat_file_path, report_date, main_directory, team_folder, employee_folder)
                if row:
                    all_data.append(row)
                    logging.debug(f"Row appended for chat file: {chat_file}")
    df = pd.DataFrame(all_data)
    #logging.debug(f"DataFrame constructed with {len(df)} rows")
    return df

# Process all team folders and chats based on the report date
all_chats_df = process_team_folders(main_directory_path, report_date)


# Save to a CSV file
csv_file_path = 'C:\\Users\\mauriceyeng\Python\\Daily-Reports\\Chat Folder from Drive\\drive-download-20231213T043134Z-001\\chat_data6.csv'  # Define your desired path and file name
all_chats_df.to_csv(csv_file_path, index=False)
#print(f"DataFrame saved as CSV at {csv_file_path}")
# Display the result
#print(all_chats_df)


2023-12-13 10:24:23,114 - DEBUG - Row appended for chat file: WhatsApp Chat with Admire EWYL22E1163.txt


ENTER THE LOOP:               datetime            sender  \
285 2023-12-12 18:12:00  +263 78 944 3351   

                                               message  
285  Here are  my Instagram and Facebook username *...  


2023-12-13 10:24:23,398 - DEBUG - Row appended for chat file: WhatsApp Chat with Allen EWYL22E0762.txt
2023-12-13 10:24:23,537 - DEBUG - Row appended for chat file: WhatsApp Chat with Alson EWYL22E0675.txt
2023-12-13 10:24:23,801 - DEBUG - Row appended for chat file: WhatsApp Chat with Anesu EWYL22E0418.txt
2023-12-13 10:24:24,023 - DEBUG - Row appended for chat file: WhatsApp Chat with Anotida EWYL23E0165.txt
2023-12-13 10:24:24,287 - DEBUG - Row appended for chat file: WhatsApp Chat with Anotidaishe EWYL23E0378.txt
2023-12-13 10:24:24,426 - DEBUG - Row appended for chat file: WhatsApp Chat with Ashley EWYL22E0726.txt
2023-12-13 10:24:24,646 - DEBUG - Row appended for chat file: WhatsApp Chat with Ashley EWYL22E1272.txt
2023-12-13 10:24:24,879 - DEBUG - Row appended for chat file: WhatsApp Chat with Ayanda EWYL23E0703.txt
2023-12-13 10:24:25,058 - DEBUG - Row appended for chat file: WhatsApp Chat with Bathandile EWYL22E0577.txt
2023-12-13 10:24:25,229 - DEBUG - Row appended for chat f

ENTER THE LOOP:               datetime            sender  \
236 2023-12-12 19:21:00  +263 78 994 5161   

                                               message  
236  Offline Application Form  Student Name:NGAVI K...  


2023-12-13 10:24:27,176 - DEBUG - Row appended for chat file: WhatsApp Chat with CHIKONDI EWYL23E0333.txt
2023-12-13 10:24:27,379 - DEBUG - Row appended for chat file: WhatsApp Chat with CHINAZVO EWYL22E0873.txt
2023-12-13 10:24:27,614 - DEBUG - Row appended for chat file: WhatsApp Chat with Colleen EWYL22E0929.txt
2023-12-13 10:24:27,753 - DEBUG - Row appended for chat file: WhatsApp Chat with Dylan EWYL23E0140.txt
2023-12-13 10:24:27,907 - DEBUG - Row appended for chat file: WhatsApp Chat with Eleanor EWYL22E0996.txt
2023-12-13 10:24:28,093 - DEBUG - Row appended for chat file: WhatsApp Chat with Ellary EWYL23E0906 (1).txt
2023-12-13 10:24:28,301 - DEBUG - Row appended for chat file: WhatsApp Chat with Glory EWYL23E0161.txt
2023-12-13 10:24:28,559 - DEBUG - Row appended for chat file: WhatsApp Chat with Hove EWYL23E0076.txt


...

ENTER THE LOOP:               datetime           sender  \
376 2023-12-12 18:43:00  +27 73 420 0758   

                                               message  
376  Good evening, I'm sorry to inform you that we ...  


2023-12-13 10:27:37,079 - DEBUG - Row appended for chat file: WhatsApp Chat with Ramona Chali_EDOOFAZAMB(12_12)SM23.txt
2023-12-13 10:27:37,219 - DEBUG - Row appended for chat file: WhatsApp Chat with Ropafadzo Khosah_EDOOFA(01_12)SM23.txt
2023-12-13 10:27:37,385 - DEBUG - Row appended for chat file: WhatsApp Chat with Rufaro Mukasa_EDOOFA(04_12)SM23.txt
2023-12-13 10:27:37,620 - DEBUG - Row appended for chat file: WhatsApp Chat with Runyararo Muchabaiwa_EDOOFA(29_11)GS.txt
2023-12-13 10:27:37,968 - DEBUG - Row appended for chat file: WhatsApp Chat with Rutendo Chitate_EDOOFA(30_10)SM23.txt
2023-12-13 10:27:38,121 - DEBUG - Row appended for chat file: WhatsApp Chat with Sean chiwundura_EDOOFA(04_12)SM23.txt
2023-12-13 10:27:38,337 - DEBUG - Row appended for chat file: WhatsApp Chat with Shallom Chimhowa_EDOOFA(20_11)SM23.txt
2023-12-13 10:27:38,601 - DEBUG - Row appended for chat file: WhatsApp Chat with Shingirai Vunganai_EA(16_11)TR.txt


ENTER THE LOOP:               datetime            sender                    message
407 2023-12-12 14:56:00  +263 77 864 0240  Hey ma'am good afternoon.


2023-12-13 10:27:38,738 - DEBUG - Row appended for chat file: WhatsApp Chat with Sibanda Siphiwe_EA(17_11)TR.txt
2023-12-13 10:27:39,003 - DEBUG - Row appended for chat file: WhatsApp Chat with Sigireta Tariro_EDOOFA(22_11)GS.txt
2023-12-13 10:27:39,262 - DEBUG - Row appended for chat file: WhatsApp Chat with Simeon Wamedzwa_EDOOFA(16_11)SM23.txt
2023-12-13 10:27:39,408 - DEBUG - Row appended for chat file: WhatsApp Chat with Simon Chikarara_EDOOFA(07_12)SM23.txt
2023-12-13 10:27:39,574 - DEBUG - Row appended for chat file: WhatsApp Chat with Simon phiri_EDOOFAZAMB(12_12)SM23.txt
2023-12-13 10:27:39,956 - DEBUG - Row appended for chat file: WhatsApp Chat with Stermon Malambo_EDOOFAZAMB(22_11)SM23.txt
2023-12-13 10:27:40,429 - DEBUG - Row appended for chat file: WhatsApp Chat with TAFADZWA MUTINDI_EDOOFA(24_11)GS.txt
2023-12-13 10:27:40,598 - DEBUG - Row appended for chat file: WhatsApp Chat with Tanaka Mushonga_EDOOFA(07_12)SM23.txt
2023-12-13 10:27:40,913 - DEBUG - Row appended for ch

2023-12-13 10:27:50,698 - DEBUG - Row appended for chat file: WhatsApp Chat with Sandra L Bako_EA(12_12)TR.txt
2023-12-13 10:27:50,815 - DEBUG - Row appended for chat file: WhatsApp Chat with SANDRA MATEMA_EA(12_12)TR.txt
2023-12-13 10:27:50,931 - DEBUG - Row appended for chat file: WhatsApp Chat with Sandra Mushawarima_EA(16_11)TR.txt
2023-12-13 10:27:51,068 - DEBUG - Row appended for chat file: WhatsApp Chat with Shayne Manyawutse_EA(16_11)TR.txt
2023-12-13 10:27:51,221 - DEBUG - Row appended for chat file: WhatsApp Chat with Shomai Prince_EA(27_10)TR.txt
2023-12-13 10:27:51,351 - DEBUG - Row appended for chat file: WhatsApp Chat with Sithole Shalom_EDOOFACHS(17_10)TR.txt
2023-12-13 10:27:51,487 - DEBUG - Row appended for chat file: WhatsApp Chat with Stephen magomana_EA(16_11)TR.txt
2023-12-13 10:27:51,600 - DEBUG - Row appended for chat file: WhatsApp Chat with Tadiwanashe Shoko_EA(12_12)TR.txt
2023-12-13 10:27:51,766 - DEBUG - Row appended for chat file: WhatsApp Chat with Tanatsw

In [ ]:
Table 4

In [6]:
import os
import pandas as pd
import datetime
import re
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Function to list all chat files in the directory structure
def list_chat_files(date_directory):
    chat_files = []
    for date_folder in os.listdir(date_directory):
        date_path = os.path.join(date_directory, date_folder)
        if os.path.isdir(date_path):
            for team_folder in os.listdir(date_path):
                team_path = os.path.join(date_path, team_folder)
                if os.path.isdir(team_path):
                    for person_folder in os.listdir(team_path):
                        person_path = os.path.join(team_path, person_folder)
                        if os.path.isdir(person_path):
                            for file in os.listdir(person_path):
                                if file.endswith('.txt'):
                                    chat_files.append(os.path.join(person_path, file))
    return chat_files

def parse_chat_file(file_path, expected_date_minus_one):
    chat_data = []
    last_non_person_time = None  # Tracks the time of the last non-person message

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            message_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*?): (.*)', line)
            system_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*)', line)
            if message_match:
                date_time_str, sender, message = message_match.groups()
            elif system_match:
                date_time_str, info = system_match.groups()
                sender = None
            else:
                continue

            date_time = pd.to_datetime(date_time_str, format='%d/%m/%y, %I:%M %p')

            if date_time.date() != expected_date_minus_one:
                continue

            if sender is not None and re.match(r'^[+\d\s-]+$', sender) is None:
                message_type = 'person'
            else:
                message_type = 'other'

            # Calculate delay
            delay = False
            if message_type == 'person' and last_non_person_time:
                diff = date_time - last_non_person_time
                delay = diff.total_seconds() > 900  # 15 minutes in seconds

            chat_data.append((date_time, sender, message_type, delay))

            if message_type == 'other':
                last_non_person_time = date_time

    return chat_data

def create_template_dataframe():
    times = [datetime.datetime(2000, 1, 1, 0, 0) + datetime.timedelta(minutes=1 * i) for i in range(1440)]
    intervals = [time.strftime('%I:%M %p') for time in times]
    df = pd.DataFrame(index=intervals)
    return df

def populate_dataframe(df, parsed_data, start_column_index):
    new_columns = {}

    for entry in parsed_data:
        date_time, sender, message_type, delay = entry
        interval_index = min((date_time.hour * 60 + date_time.minute) // 1, 1439)
        interval = df.index[interval_index]

        if start_column_index not in new_columns:
            new_columns[start_column_index] = pd.Series(0, index=df.index)  # For 'person'
        if start_column_index + 1 not in new_columns:
            new_columns[start_column_index + 1] = pd.Series(0, index=df.index)  # For 'other'
        if start_column_index + 2 not in new_columns:
            new_columns[start_column_index + 2] = pd.Series(False, index=df.index)  # For delay column

        if message_type == 'person':
            new_columns[start_column_index].at[interval] = 1
        elif message_type == 'other':
            new_columns[start_column_index + 1].at[interval] = 1

        new_columns[start_column_index + 2].at[interval] = delay  # Set delay flag

    df = pd.concat([df, pd.DataFrame(new_columns)], axis=1)
    return df, start_column_index + 3

def process_person_chats(chat_files):
    dataframes = {}
    for file in chat_files:
        parts = file.split(os.sep)
        date_folder, person = parts[-4], parts[-2]

        try:
            folder_date = pd.to_datetime(date_folder, format='%Y-%m-%d').date()
        except ValueError:
            continue

        expected_date_minus_one = folder_date - datetime.timedelta(days=1)
        key = f"{folder_date.strftime('%Y-%m-%d')}_{person}"

        if key not in dataframes:
            dataframes[key] = create_template_dataframe()
            start_column_index = 0
        else:
            if not dataframes[key].columns.empty:
                start_column_index = max(dataframes[key].columns) + 1
            else:
                start_column_index = 0

        parsed_data = parse_chat_file(file, expected_date_minus_one)
        dataframes[key], start_column_index = populate_dataframe(dataframes[key], parsed_data, start_column_index)

    return dataframes

def create_graphs(df, person_identifier, base_directory):
    graph_directory = os.path.join(base_directory, "Graphs")
    os.makedirs(graph_directory, exist_ok=True)

    # Sum the values for 'person' and 'other' messages for each minute
    person_chat_activity = df.iloc[:, 0::3].sum(axis=1)
    other_chat_activity = df.iloc[:, 1::3].sum(axis=1)
    
    # Find the first and last non-zero indices for chats
    non_zero_indices = person_chat_activity[person_chat_activity > 0].index
    first_chat_time = non_zero_indices[0] if not non_zero_indices.empty else df.index[0]
    last_chat_time = non_zero_indices[-1] if not non_zero_indices.empty else df.index[-1]

    # Creating the plot
    fig, ax = plt.subplots(figsize=(30, 10))
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')
    ax.xaxis.label.set_color('black')
    ax.yaxis.label.set_color('black')
    ax.title.set_color('black')
    ax.tick_params(axis='x', colors='black')
    ax.tick_params(axis='y', colors='black')


    # Plot the bar for 'person' messages
    ax.bar(df.index, person_chat_activity, color='lime', width=2, label='Counselor')

    # Plot the bar for 'other' messages, stacked on top of 'person' messages
    #ax.bar(df.index, other_chat_activity, color='darkgreen', width=1, label='Other Messages', bottom=person_chat_activity)
    ax.plot(df.index, other_chat_activity, color='darkgreen', linestyle=':', label='Student')
    # Draw white lines for the axes
    ax.axhline(0, color='black', linewidth=3)  # Changed color to black
    ax.axvline(first_chat_time, color='white', linewidth=3)
    #ax.axvline(df.index[0], color='black', linewidth=3)  # Changed color to black


    # Rotate x-axis labels to prevent overlap and increase label font sizes
    plt.xticks(rotation=90, fontsize=12)

    # Set y-axis to show every integer tick
    plt.yticks(np.arange(0, 11, 1), fontsize=12)

    # Set x-axis to show the range from the first chat to the last chat
    ax.set_xlim(first_chat_time, last_chat_time)
    #ax.set_xlim(df.index[0], df.index[-1])
    ax.xaxis.set_major_locator(ticker.MaxNLocator(96))  # Set locator for 15-minute intervals

    # Set y-axis dynamic range based on the maximum chat activity with a buffer
    max_activity = max(person_chat_activity.max(), other_chat_activity.max())
    ax.set_ylim(0, 11)

    # Increasing font size for labels and title
    ax.set_xlabel('Time', fontsize=12)
    ax.set_ylabel('Number of Chats', fontsize=12)
    ax.set_title(f'Chat Activity for {person_identifier}', fontsize=14)

    # Create and set the legend
    legend = ax.legend(facecolor='lightgray', edgecolor='black', fontsize=12, fancybox=True)
    for text in legend.get_texts():
        text.set_color('black')
        text.set_weight('bold')

    # Saving the graph
    graph_file_name = f"{person_identifier}.png"
    plt.savefig(os.path.join(graph_directory, graph_file_name), format='png', dpi=300, bbox_inches='tight')
    print(f"Graph saved as {graph_file_name}")

    plt.close(fig)


# Main script
date_directory = "C:\\Users\\mauriceyeng\Python\\Daily-Reports\\Chat Folder from Drive\\drive-download-20231213T043134Z-001"
chat_files = list_chat_files(date_directory)
person_dataframes = process_person_chats(chat_files)

for person_identifier, df in person_dataframes.items():
    create_graphs(df, person_identifier, date_directory)

2023-12-13 10:40:45,190 - DEBUG - matplotlib data path: C:\Users\mauriceyeng\anaconda3\Lib\site-packages\matplotlib\mpl-data
2023-12-13 10:40:45,190 - DEBUG - CONFIGDIR=C:\Users\mauriceyeng\.matplotlib
2023-12-13 10:40:45,200 - DEBUG - interactive is False
2023-12-13 10:40:45,201 - DEBUG - platform is win32
2023-12-13 10:40:45,278 - DEBUG - CACHEDIR=C:\Users\mauriceyeng\.matplotlib
2023-12-13 10:40:45,278 - DEBUG - Using fontManager instance from C:\Users\mauriceyeng\.matplotlib\fontlist-v330.json
2023-12-13 10:41:17,414 - DEBUG - Loaded backend module://matplotlib_inline.backend_inline version unknown.
2023-12-13 10:41:17,417 - DEBUG - Loaded backend module://matplotlib_inline.backend_inline version unknown.
2023-12-13 10:41:17,420 - DEBUG - findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0.
2023-12-13 10:41:17,421 - DEBUG - findfont: score(FontEntry(fname='C:\\Users\\mauriceyeng\\anaconda3\\Lib\\site-packages\\matplotlib\\mpl-data\\font

2023-12-13 10:41:17,435 - DEBUG - findfont: score(FontEntry(fname='C:\\Users\\mauriceyeng\\anaconda3\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\STIXNonUniBolIta.ttf', name='STIXNonUnicode', style='italic', variant='normal', weight=700, stretch='normal', size='scalable')) = 11.335
2023-12-13 10:41:17,435 - DEBUG - findfont: score(FontEntry(fname='C:\\Users\\mauriceyeng\\anaconda3\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\cmmi10.ttf', name='cmmi10', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
2023-12-13 10:41:17,436 - DEBUG - findfont: score(FontEntry(fname='C:\\Users\\mauriceyeng\\anaconda3\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\DejaVuSansMono-BoldOblique.ttf', name='DejaVu Sans Mono', style='oblique', variant='normal', weight=700, stretch='normal', size='scalable')) = 11.335
2023-12-13 10:41:17,436 - DEBUG - findfont: score(FontEntry(fname='C:\\Users\\mauriceyeng\\anaconda3\\Lib\\site-packages\\matpl

2023-12-13 10:41:17,452 - DEBUG - findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\trebucbd.ttf', name='Trebuchet MS', style='normal', variant='normal', weight=700, stretch='normal', size='scalable')) = 10.335
2023-12-13 10:41:17,453 - DEBUG - findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\COOPBL.TTF', name='Cooper Black', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
2023-12-13 10:41:17,453 - DEBUG - findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\ITCEDSCR.TTF', name='Edwardian Script ITC', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
2023-12-13 10:41:17,454 - DEBUG - findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\VIVALDII.TTF', name='Vivaldi', style='italic', variant='normal', weight=400, stretch='normal', size='scalable')) = 11.05
2023-12-13 10:41:17,455 - DEBUG - findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\LCALLIG.TTF', name='Lucida Calligraphy', style='italic', va

2023-12-13 10:41:17,469 - DEBUG - findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\LHANDW.TTF', name='Lucida Handwriting', style='italic', variant='normal', weight=400, stretch='normal', size='scalable')) = 11.05
2023-12-13 10:41:17,469 - DEBUG - findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\phagspa.ttf', name='Microsoft PhagsPa', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
2023-12-13 10:41:17,469 - DEBUG - findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\ebrima.ttf', name='Ebrima', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
2023-12-13 10:41:17,469 - DEBUG - findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\micross.ttf', name='Microsoft Sans Serif', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
2023-12-13 10:41:17,475 - DEBUG - findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\consola.ttf', name='Consolas', style='normal', variant

2023-12-13 10:41:17,489 - DEBUG - findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\BOD_I.TTF', name='Bodoni MT', style='italic', variant='normal', weight=400, stretch='normal', size='scalable')) = 11.05
2023-12-13 10:41:17,489 - DEBUG - findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\segmdl2.ttf', name='Segoe MDL2 Assets', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
2023-12-13 10:41:17,489 - DEBUG - findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\ARIALNI.TTF', name='Arial', style='italic', variant='normal', weight=400, stretch='condensed', size='scalable')) = 7.613636363636363
2023-12-13 10:41:17,489 - DEBUG - findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\YuGothM.ttc', name='Yu Gothic', style='normal', variant='normal', weight=500, stretch='normal', size='scalable')) = 10.145
2023-12-13 10:41:17,489 - DEBUG - findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\verdanaz.ttf', name='Verdana', style='italic', variant='nor

...

Graph saved as 2023-12-13_Shashwat Edoofa 2.png


2023-12-13 10:43:11,349 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
2023-12-13 10:43:11,438 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
2023-12-13 10:43:11,444 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
2023-12-13 10:43:11,445 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
2023-12-13 10:43:13,973 - INFO - Using categorical units to plot a list of strings that are all parsable as 

Graph saved as 2023-12-13_Shubham Madhwal.png


2023-12-13 10:43:18,835 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
2023-12-13 10:43:18,923 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
2023-12-13 10:43:18,929 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
2023-12-13 10:43:18,930 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
2023-12-13 10:43:20,462 - INFO - Using categorical units to plot a list of strings that are all parsable as 

Graph saved as 2023-12-13_Tushti.png
